In [ ]:
# Required Libraries
import folium
import math
import ipywidgets as widgets
from IPython.display import display, clear_output
from geopy.distance import geodesic
from folium.plugins import MeasureControl

# Function to plot locations on a map
def plot_on_map(latitudes, longitudes, names):
    m = folium.Map(location=[sum(latitudes)/len(latitudes), sum(longitudes)/len(longitudes)], zoom_start=5)
    for lat, lon, name in zip(latitudes, longitudes, names):
        folium.Marker([lat, lon], tooltip=name).add_to(m)
    return m

# Function to calculate distance between two locations
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371.0
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = R * c
    return distance

# Interactive Map Class
class InteractiveMap:
    def __init__(self, latitudes, longitudes, names):
        self.map = folium.Map(location=[sum(latitudes)/len(latitudes), sum(longitudes)/len(longitudes)], zoom_start=5)
        self.selected_points = []
        for lat, lon, name in zip(latitudes, longitudes, names):
            marker = folium.Marker([lat, lon], popup=name, tooltip=name)
            marker.add_to(self.map)
        self.map.add_child(folium.ClickForMarker(popup='Selected Point'))

    def display_map(self):
        display(self.map)

    def get_distance(self, b1, b2):
        point1 = (b1.location[0], b1.location[1])
        point2 = (b2.location[0], b2.location[1])
        distance = geodesic(point1, point2).kilometers
        return distance

    def on_button_click(self, b):
        if b.description == 'Select Point 1':
            self.selected_points.append(self.map.location)
            b.description = 'Point 1 Selected'
        elif b.description == 'Select Point 2':
            self.selected_points.append(self.map.location)
            b.description = 'Point 2 Selected'
            distance = self.get_distance(self.selected_points[0], self.selected_points[1])
            print(f'Distance between the two points: {distance:.2f} km')
            self.selected_points = []

# Function to create an interactive map with measurement tool
def create_interactive_map(latitudes, longitudes, names):
    m = folium.Map(location=[sum(latitudes)/len(latitudes), sum(longitudes)/len(longitudes)], zoom_start=5)
    for lat, lon, name in zip(latitudes, longitudes, names):
        folium.Marker([lat, lon], popup=name, tooltip=name).add_to(m)
    m.add_child(MeasureControl(primary_length_unit='kilometers'))
    return m

# Dropdown function to calculate distance
def calculate_distance_dropdown(change):
    if dropdown1.value and dropdown2.value:
        loc1 = locations[dropdown1.value]
        loc2 = locations[dropdown2.value]
        distance = geodesic(loc1, loc2).kilometers
        output.clear_output(wait=True)
        with output:
            print(f'Distance between {dropdown1.value} and {dropdown2.value}: {distance:.2f} km')

# Locations data
locations = {
    'ECAC001':(26.8952758,83.4864332),
'ECAC002':(26.6996666,84.1857677),
'ECAC003':(26.7507434,83.741661),
'ECAC004':(28.0736129,80.4710711),
}

# Dropdown widgets
dropdown1 = widgets.Dropdown(options=[''] + list(locations.keys()), description='Location 1:')
dropdown2 = widgets.Dropdown(options=[''] + list(locations.keys()), description='Location 2:')
dropdown1.observe(calculate_distance_dropdown, names='value')
dropdown2.observe(calculate_distance_dropdown, names='value')
output = widgets.Output()

# Display map with dropdowns
m = folium.Map(location=[sum([loc[0] for loc in locations.values()])/len(locations),
                         sum([loc[1] for loc in locations.values()])/len(locations)], zoom_start=5)
for name, (lat, lon) in locations.items():
    folium.Marker([lat, lon], popup=name, tooltip=name).add_to(m)
display(dropdown1, dropdown2, output, m)